In [6]:
import numpy as np
from scipy.linalg import solve

def feasibility(A, b, a_list, b_list, x0, s0, mu_init=1.0, tol=1e-6, max_iter=50):

    n = x0.shape[0]
    m = len(a_list)
    x = x0.copy()
    s = s0
    mu = mu_init
    
    for it in range(max_iter):
        # Construct the barrier objective
        g = np.array([a_i @ x + b_i - s for a_i, b_i in zip(a_list, b_list)])
        if np.any(g >= 0):
            raise ValueError("Infeasible starting point! g(x, s) must be strictly negative.")

        # Gradient of objective
        grad = np.zeros(n + 1)  # [grad_x, grad_s]
        for i in range(m):
            ai = a_list[i]
            bi = b_list[i]
            denom = s - (ai @ x + bi)
            grad[:n] += (mu / denom) * ai
            grad[n] += (-mu / denom)
        grad[n] += 1  # derivative of s in objective

        # Hessian 
        hess = np.zeros((n + 1, n + 1))
        for i in range(m):
            ai = a_list[i]
            denom = s - (ai @ x + b_list[i])
            ai = ai.reshape(-1,1)
            hess[:n, :n] += (mu / denom**2) * (ai @ ai.T)
            hess[:n, n] += (-mu / denom**2) * ai.flatten()
            hess[n, :n] += (-mu / denom**2) * ai.flatten()
            hess[n, n] += (mu / denom**2)

        # KKT system 
        p = A.shape[0]  # number of equality constraints
        KKT_matrix = np.block([
            [hess, np.vstack([A.T, np.zeros((1, p))])],
            [np.hstack([A, np.zeros((p, 1))]), np.zeros((p, p))]
        ])
        KKT_rhs = -np.hstack((grad, np.zeros(p)))


        # Solve for Newton step
        delta = solve(KKT_matrix, KKT_rhs)
        dx = delta[:n]
        ds = delta[n]
        _ = delta[n+1:]

        # Line search
        alpha = 1.0
        beta = 0.9
        while True:
            new_x = x + alpha * dx
            new_s = s + alpha * ds
            new_g = np.array([a_i @ new_x + b_i - new_s for a_i, b_i in zip(a_list, b_list)])
            if np.all(new_g < 0):
                break
            alpha *= beta

        # Update
        x += alpha * dx
        s += alpha * ds

        # Checking the convergence
        if np.linalg.norm(grad) < tol:
            break

        # Decrease mu
        mu *= 0.5

    return x, s




import numpy as np

# Equality constraint: x1 + x2 = 1
eq_A = np.array([[1, 1]])
eq_b = np.array([1])

# Inequality constraints: x1 >= 0, x2 >= 0
ineq_A_list = [
    np.array([-1, 0]),  # -x1 <= 0  ⟹  x1 >= 0
    np.array([0, -1])   # -x2 <= 0  ⟹  x2 >= 0
]
ineq_b_list = [0, 0]

# Initial point and slack variable
x_init = np.array([0.6, 0.7])
slack_init = 1.0  # Should be larger than the max violation of constraints at x_init

# Run Phase I feasibility check
x_opt, slack_opt = feasibility(eq_A, eq_b, ineq_A_list, ineq_b_list, x_init, slack_init)

# Print results
print("Optimal x:", x_opt)
print("Optimal slack s:", slack_opt)

# Interpret and print feasibility result
if slack_opt < 0:
    result_msg = "Result: The linear program is strictly feasible."
elif slack_opt > 0:
    result_msg = "Result: The linear program is infeasible."
else:
    result_msg = "Result: The linear program is feasible but not strictly feasible."

print(result_msg)



Optimal x: [0.65 0.65]
Optimal slack s: -0.6499999999999996
Result: The linear program is strictly feasible.


/home/sshabani/.pyenv/versions/SAM_env/lib/python3.7/site-packages/ipykernel_launcher.py:49: LinAlgWarning: Ill-conditioned matrix (rcond=2.08106e-17): result may not be accurate.
/home/sshabani/.pyenv/versions/SAM_env/lib/python3.7/site-packages/ipykernel_launcher.py:49: LinAlgWarning: Ill-conditioned matrix (rcond=9.24224e-18): result may not be accurate.
